In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import mysql.connector
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,MultiPolygon,LineString,MultiLineString,LinearRing,Point
from shapely import symmetric_difference
from shapely.ops import nearest_points
from shapely.wkt import loads
import scipy
import time
import numpy as np
import random
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import osmnx as ox
import glob
from itertools import combinations,product
from copy import deepcopy
import os
from lib_contour_lines import *
import mysql.connector


In [2]:

cnx = mysql.connector.connect(user='spirz', 
                              password='this_is_my_PASSWORD_m8',
                              host='localhost',
                              database='dem_from_contours_lines',
                              autocommit=True)
cursor = cnx.cursor(buffered=True,dictionary=True)
contours_lines_table_name='contours_lines'
tree_edges_table_name="tree_edges"
osm_nodes_table_name="osm_nodes"
osm_intersections_table_name="osm_intersections"
2

2

In [3]:
t1=time.time()
G=build_graph(cursor)
t2=time.time()
print(t2-t1)

0.6398584842681885


In [9]:
t1=time.time()
cmd="SELECT osm_id,contour_line_id,ST_asText(geometry) AS geometry FROM %s WHERE contour_line_id IS NOT NULL"%osm_nodes_table_name
cursor.execute(cmd)
nodes_data=pd.DataFrame(cursor.fetchall())
nodes_data['geometry']=nodes_data['geometry'].apply(lambda pt:loads(pt))
nodes_data=nodes_data.set_index('osm_id',drop=False)
nodes_data=gpd.GeoDataFrame(nodes_data,geometry='geometry',crs='epsg:4326')
nodes_data['old_geometry']=nodes_data['geometry'].copy()
crs=nodes_data.estimate_utm_crs()
nodes_data=nodes_data.to_crs(crs)
nodes_data['depth']=nodes_data['contour_line_id'].apply(lambda contour_line_id:G.nodes()[contour_line_id]['depth'])
t2=time.time()
print(t2-t1)

1.5327253341674805


In [ ]:
max_nb_neighbors=float('inf')
t1=time.time()
nodes_data['elevation']=None
contour_line_ids=nodes_data['contour_line_id'].unique()
contour_lines_ids=sorted(contour_line_ids,key=lambda contour_line_id:len(list(G.neighbors(contour_line_id))))
contour_lines_ids=contour_lines_ids[::-1]
print('sorted')
for k,root_contour_line_id in enumerate(contour_lines_ids):
    if (k+1)%10==0:
        print('%i/%i'%(k+1,len(contour_line_ids)))
    neighbor_contour_line_ids=list(G.neighbors(root_contour_line_id))
    print('%i neighbors'%len(neighbor_contour_line_ids))
    if 0<len(neighbor_contour_line_ids)<max_nb_neighbors:
        t3=time.time()
        local_contours_line_ids=[root_contour_line_id]+neighbor_contour_line_ids
        local_contour_data=get_nodes_data(cursor,local_contours_line_ids)
        local_contour_data=local_contour_data.to_crs(crs)
        local_contour_data=local_contour_data.set_index('id')
        t4=time.time()
        print('setting up df took %f'%(t4-t3))
        local_nodes_data=nodes_data[nodes_data.contour_line_id==root_contour_line_id].copy()
        local_nodes_data['elevation']=local_nodes_data['geometry'].apply(lambda osm_pt:get_osm_node_elevation(osm_pt,local_contour_data))
        nodes_data.loc[nodes_data.contour_line_id==root_contour_line_id,'elevation']=local_nodes_data.loc[:,'elevation']
    
        t5=time.time()
        print('finding affine basis took %f'%(t5-t4))
        print('intermediate time %f'%(t5-t1))
t2=time.time()
print(t2-t1)

sorted
607 neighbors
setting up df took 0.796401


/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


In [7]:
nodes_data=nodes_data[nodes_data.elevation==nodes_data.elevation]
diffs=nodes_data['elevation']-nodes_data['contour_line_id'].apply(lambda node:G.nodes()[node]['elevation'])
np.min(diffs),np.mean(diffs),np.median(diffs),np.max(diffs)
                                                                  

(-2.4193548387096797, 3.03846418619785, 3.3380884450784833, 4.0)

In [10]:
len(nodes_data)

83512